In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/star/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 500
test_batch = 500
lr = 0.04
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b0/to_pggan/2000shot/intracm' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.4, 'drop_connect_rate':0.4})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/star/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
loss_list = []
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    loss_list.append([train_loss, test_loss, source_loss])
    
    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    if (epoch+1) % 50 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.040000
Train | 8/8 | Loss:1.2923 | MainLoss:1.0577 | Alpha:0.1115 | SPLoss:0.2302 | CLSLoss:1.8752 | top1:58.8000 | AUROC:0.0000
Test | 65/8 | Loss:0.5736 | MainLoss:0.5736 | SPLoss:0.2824 | CLSLoss:1.7571 | top1:75.0623 | AUROC:0.8642
Test | 62/8 | Loss:0.3978 | MainLoss:0.3978 | SPLoss:0.2824 | CLSLoss:1.7571 | top1:87.7654 | AUROC:0.9943

Epoch: [2 | 1000] LR: 0.052000
Train | 8/8 | Loss:0.7881 | MainLoss:0.5608 | Alpha:0.1157 | SPLoss:0.2740 | CLSLoss:1.6920 | top1:74.1250 | AUROC:0.0000
Test | 65/8 | Loss:0.4111 | MainLoss:0.4111 | SPLoss:0.2668 | CLSLoss:1.6110 | top1:85.3364 | AUROC:0.9526
Test | 62/8 | Loss:0.3395 | MainLoss:0.3395 | SPLoss:0.2668 | CLSLoss:1.6110 | top1:86.4613 | AUROC:0.9923

Epoch: [3 | 1000] LR: 0.064000
Train | 8/8 | Loss:0.6900 | MainLoss:0.4855 | Alpha:0.1124 | SPLoss:0.2813 | CLSLoss:1.5398 | top1:77.5750 | AUROC:0.0000
Test | 65/8 | Loss:0.2682 | MainLoss:0.2682 | SPLoss:0.3055 | CLSLoss:1.4468 | top1:92.7414 | AUROC:0.9876
Tes

Test | 65/8 | Loss:0.2468 | MainLoss:0.2468 | SPLoss:0.4758 | CLSLoss:0.1600 | top1:90.9875 | AUROC:0.9974
Test | 62/8 | Loss:0.1995 | MainLoss:0.1995 | SPLoss:0.4758 | CLSLoss:0.1600 | top1:92.4410 | AUROC:0.9942

Epoch: [24 | 1000] LR: 0.159943
Train | 8/8 | Loss:0.4936 | MainLoss:0.4144 | Alpha:0.1152 | SPLoss:0.5456 | CLSLoss:0.1397 | top1:81.8000 | AUROC:0.0000
Test | 65/8 | Loss:0.2578 | MainLoss:0.2578 | SPLoss:0.4901 | CLSLoss:0.1503 | top1:89.8785 | AUROC:0.9975
Test | 62/8 | Loss:0.2104 | MainLoss:0.2104 | SPLoss:0.4901 | CLSLoss:0.1503 | top1:93.3552 | AUROC:0.9935

Epoch: [25 | 1000] LR: 0.159933
Train | 8/8 | Loss:0.4568 | MainLoss:0.3803 | Alpha:0.1187 | SPLoss:0.4995 | CLSLoss:0.1447 | top1:83.5750 | AUROC:0.0000
Test | 65/8 | Loss:0.1163 | MainLoss:0.1163 | SPLoss:0.5224 | CLSLoss:0.1500 | top1:98.4143 | AUROC:0.9987
Test | 62/8 | Loss:0.4899 | MainLoss:0.4899 | SPLoss:0.5224 | CLSLoss:0.1500 | top1:72.0839 | AUROC:0.9926

Epoch: [26 | 1000] LR: 0.159923
Train | 8/8 | L

Test | 62/8 | Loss:0.5051 | MainLoss:0.5051 | SPLoss:0.6718 | CLSLoss:0.1217 | top1:68.6697 | AUROC:0.9761

Epoch: [46 | 1000] LR: 0.159544
Train | 8/8 | Loss:0.4662 | MainLoss:0.3766 | Alpha:0.1204 | SPLoss:0.6009 | CLSLoss:0.1414 | top1:83.4500 | AUROC:0.0000
Test | 65/8 | Loss:0.1431 | MainLoss:0.1431 | SPLoss:0.5487 | CLSLoss:0.1585 | top1:97.2586 | AUROC:0.9992
Test | 62/8 | Loss:0.3584 | MainLoss:0.3584 | SPLoss:0.5487 | CLSLoss:0.1585 | top1:81.9299 | AUROC:0.9917

Epoch: [47 | 1000] LR: 0.159517
Train | 8/8 | Loss:0.4397 | MainLoss:0.3639 | Alpha:0.1087 | SPLoss:0.5359 | CLSLoss:0.1616 | top1:84.0750 | AUROC:0.0000
Test | 65/8 | Loss:0.1342 | MainLoss:0.1342 | SPLoss:0.5176 | CLSLoss:0.1648 | top1:97.0499 | AUROC:0.9983
Test | 62/8 | Loss:0.3520 | MainLoss:0.3520 | SPLoss:0.5176 | CLSLoss:0.1648 | top1:82.6081 | AUROC:0.9937

Epoch: [48 | 1000] LR: 0.159489
Train | 8/8 | Loss:0.4502 | MainLoss:0.3730 | Alpha:0.1148 | SPLoss:0.5109 | CLSLoss:0.1613 | top1:82.8750 | AUROC:0.0000


Train | 8/8 | Loss:0.4006 | MainLoss:0.3601 | Alpha:0.4073 | SPLoss:0.0194 | CLSLoss:0.0799 | top1:84.5250 | AUROC:0.0000
Test | 65/8 | Loss:0.1656 | MainLoss:0.1656 | SPLoss:0.0196 | CLSLoss:0.0797 | top1:96.6511 | AUROC:0.9988
Test | 62/8 | Loss:0.3180 | MainLoss:0.3180 | SPLoss:0.0196 | CLSLoss:0.0797 | top1:86.2713 | AUROC:0.9933

Epoch: [69 | 1000] LR: 0.015872
Train | 8/8 | Loss:0.4013 | MainLoss:0.3610 | Alpha:0.4095 | SPLoss:0.0193 | CLSLoss:0.0790 | top1:84.4000 | AUROC:0.0000
Test | 65/8 | Loss:0.1697 | MainLoss:0.1697 | SPLoss:0.0195 | CLSLoss:0.0793 | top1:96.4673 | AUROC:0.9987
Test | 62/8 | Loss:0.3104 | MainLoss:0.3104 | SPLoss:0.0195 | CLSLoss:0.0793 | top1:86.8250 | AUROC:0.9935

Epoch: [70 | 1000] LR: 0.015868
Train | 8/8 | Loss:0.3979 | MainLoss:0.3572 | Alpha:0.4100 | SPLoss:0.0197 | CLSLoss:0.0795 | top1:84.8750 | AUROC:0.0000
Test | 65/8 | Loss:0.1717 | MainLoss:0.1717 | SPLoss:0.0199 | CLSLoss:0.0792 | top1:96.2866 | AUROC:0.9987
Test | 62/8 | Loss:0.3047 | MainL

Test | 65/8 | Loss:0.1730 | MainLoss:0.1730 | SPLoss:0.0242 | CLSLoss:0.0779 | top1:96.1308 | AUROC:0.9987
Test | 62/8 | Loss:0.3025 | MainLoss:0.3025 | SPLoss:0.0242 | CLSLoss:0.0779 | top1:87.1691 | AUROC:0.9940

Epoch: [91 | 1000] LR: 0.015755
Train | 8/8 | Loss:0.3875 | MainLoss:0.3451 | Alpha:0.4116 | SPLoss:0.0246 | CLSLoss:0.0784 | top1:85.4250 | AUROC:0.0000
Test | 65/8 | Loss:0.1767 | MainLoss:0.1767 | SPLoss:0.0245 | CLSLoss:0.0782 | top1:95.9065 | AUROC:0.9987
Test | 62/8 | Loss:0.2949 | MainLoss:0.2949 | SPLoss:0.0245 | CLSLoss:0.0782 | top1:87.7228 | AUROC:0.9938

Epoch: [92 | 1000] LR: 0.015749
Train | 8/8 | Loss:0.3895 | MainLoss:0.3469 | Alpha:0.4132 | SPLoss:0.0247 | CLSLoss:0.0782 | top1:85.3500 | AUROC:0.0000
Test | 65/8 | Loss:0.1814 | MainLoss:0.1814 | SPLoss:0.0249 | CLSLoss:0.0789 | top1:95.4922 | AUROC:0.9986
Test | 62/8 | Loss:0.2832 | MainLoss:0.2832 | SPLoss:0.0249 | CLSLoss:0.0789 | top1:88.5190 | AUROC:0.9937

Epoch: [93 | 1000] LR: 0.015742
Train | 8/8 | L

...

Train | 8/8 | Loss:0.3686 | MainLoss:0.3394 | Alpha:0.4181 | SPLoss:0.0003 | CLSLoss:0.0694 | top1:85.5000 | AUROC:0.0000
Test | 65/8 | Loss:0.1680 | MainLoss:0.1680 | SPLoss:0.0004 | CLSLoss:0.0689 | top1:95.9782 | AUROC:0.9991
Test | 62/8 | Loss:0.2757 | MainLoss:0.2757 | SPLoss:0.0004 | CLSLoss:0.0689 | top1:88.7353 | AUROC:0.9918

Epoch: [203 | 1000] LR: 0.014602
Train | 8/8 | Loss:0.3508 | MainLoss:0.3212 | Alpha:0.4218 | SPLoss:0.0006 | CLSLoss:0.0695 | top1:86.4500 | AUROC:0.0000
Test | 65/8 | Loss:0.1704 | MainLoss:0.1704 | SPLoss:0.0008 | CLSLoss:0.0709 | top1:95.6137 | AUROC:0.9990
Test | 62/8 | Loss:0.2649 | MainLoss:0.2649 | SPLoss:0.0008 | CLSLoss:0.0709 | top1:89.3807 | AUROC:0.9914

Epoch: [204 | 1000] LR: 0.014588
Train | 8/8 | Loss:0.3565 | MainLoss:0.3262 | Alpha:0.4206 | SPLoss:0.0009 | CLSLoss:0.0711 | top1:86.5500 | AUROC:0.0000
Test | 65/8 | Loss:0.1674 | MainLoss:0.1674 | SPLoss:0.0011 | CLSLoss:0.0708 | top1:95.6916 | AUROC:0.9991
Test | 62/8 | Loss:0.2696 | Mai

Test | 65/8 | Loss:0.1745 | MainLoss:0.1745 | SPLoss:0.0082 | CLSLoss:0.0698 | top1:95.2337 | AUROC:0.9991
Test | 62/8 | Loss:0.2502 | MainLoss:0.2502 | SPLoss:0.0082 | CLSLoss:0.0698 | top1:90.3735 | AUROC:0.9911

Epoch: [225 | 1000] LR: 0.014275
Train | 8/8 | Loss:0.3583 | MainLoss:0.3257 | Alpha:0.4198 | SPLoss:0.0082 | CLSLoss:0.0694 | top1:86.4750 | AUROC:0.0000
Test | 65/8 | Loss:0.1526 | MainLoss:0.1526 | SPLoss:0.0083 | CLSLoss:0.0700 | top1:96.2523 | AUROC:0.9992
Test | 62/8 | Loss:0.2757 | MainLoss:0.2757 | SPLoss:0.0083 | CLSLoss:0.0700 | top1:88.7320 | AUROC:0.9909

Epoch: [226 | 1000] LR: 0.014259
Train | 8/8 | Loss:0.3488 | MainLoss:0.3159 | Alpha:0.4209 | SPLoss:0.0084 | CLSLoss:0.0698 | top1:86.8250 | AUROC:0.0000
Test | 65/8 | Loss:0.1610 | MainLoss:0.1610 | SPLoss:0.0086 | CLSLoss:0.0701 | top1:95.8318 | AUROC:0.9992
Test | 62/8 | Loss:0.2633 | MainLoss:0.2633 | SPLoss:0.0086 | CLSLoss:0.0701 | top1:89.5249 | AUROC:0.9910

Epoch: [227 | 1000] LR: 0.014243
Train | 8/8 

Test | 62/8 | Loss:0.2509 | MainLoss:0.2509 | SPLoss:0.0119 | CLSLoss:0.0679 | top1:90.4686 | AUROC:0.9905

Epoch: [247 | 1000] LR: 0.013917
Train | 8/8 | Loss:0.3585 | MainLoss:0.3249 | Alpha:0.4201 | SPLoss:0.0118 | CLSLoss:0.0680 | top1:86.2250 | AUROC:0.0000
Test | 65/8 | Loss:0.1598 | MainLoss:0.1598 | SPLoss:0.0119 | CLSLoss:0.0689 | top1:95.8816 | AUROC:0.9992
Test | 62/8 | Loss:0.2641 | MainLoss:0.2641 | SPLoss:0.0119 | CLSLoss:0.0689 | top1:89.4856 | AUROC:0.9905

Epoch: [248 | 1000] LR: 0.013900
Train | 8/8 | Loss:0.3588 | MainLoss:0.3249 | Alpha:0.4190 | SPLoss:0.0120 | CLSLoss:0.0688 | top1:86.4000 | AUROC:0.0000
Test | 65/8 | Loss:0.1611 | MainLoss:0.1611 | SPLoss:0.0121 | CLSLoss:0.0690 | top1:95.8193 | AUROC:0.9992
Test | 62/8 | Loss:0.2626 | MainLoss:0.2626 | SPLoss:0.0121 | CLSLoss:0.0690 | top1:89.5347 | AUROC:0.9908

Epoch: [249 | 1000] LR: 0.013883
Train | 8/8 | Loss:0.3663 | MainLoss:0.3326 | Alpha:0.4172 | SPLoss:0.0120 | CLSLoss:0.0689 | top1:85.7500 | AUROC:0.00

Train | 8/8 | Loss:0.3461 | MainLoss:0.3167 | Alpha:0.4236 | SPLoss:0.0004 | CLSLoss:0.0690 | top1:86.4500 | AUROC:0.0000
Test | 65/8 | Loss:0.1569 | MainLoss:0.1569 | SPLoss:0.0004 | CLSLoss:0.0691 | top1:96.0000 | AUROC:0.9992
Test | 62/8 | Loss:0.2652 | MainLoss:0.2652 | SPLoss:0.0004 | CLSLoss:0.0691 | top1:89.3218 | AUROC:0.9906

Epoch: [270 | 1000] LR: 0.001351
Train | 8/8 | Loss:0.3443 | MainLoss:0.3148 | Alpha:0.4237 | SPLoss:0.0004 | CLSLoss:0.0691 | top1:86.9500 | AUROC:0.0000
Test | 65/8 | Loss:0.1566 | MainLoss:0.1566 | SPLoss:0.0004 | CLSLoss:0.0691 | top1:96.0062 | AUROC:0.9992
Test | 62/8 | Loss:0.2657 | MainLoss:0.2657 | SPLoss:0.0004 | CLSLoss:0.0691 | top1:89.2955 | AUROC:0.9907

Epoch: [271 | 1000] LR: 0.001349
Train | 8/8 | Loss:0.3517 | MainLoss:0.3224 | Alpha:0.4220 | SPLoss:0.0004 | CLSLoss:0.0691 | top1:86.5000 | AUROC:0.0000
Test | 65/8 | Loss:0.1542 | MainLoss:0.1542 | SPLoss:0.0004 | CLSLoss:0.0690 | top1:96.1371 | AUROC:0.9992
Test | 62/8 | Loss:0.2690 | Mai

Test | 65/8 | Loss:0.1549 | MainLoss:0.1549 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:96.0903 | AUROC:0.9992
Test | 62/8 | Loss:0.2671 | MainLoss:0.2671 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:89.1612 | AUROC:0.9906

Epoch: [292 | 1000] LR: 0.001310
Train | 8/8 | Loss:0.3444 | MainLoss:0.3148 | Alpha:0.4237 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:87.1500 | AUROC:0.0000
Test | 65/8 | Loss:0.1547 | MainLoss:0.1547 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:96.0841 | AUROC:0.9993
Test | 62/8 | Loss:0.2671 | MainLoss:0.2671 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:89.1579 | AUROC:0.9906

Epoch: [293 | 1000] LR: 0.001308
Train | 8/8 | Loss:0.3477 | MainLoss:0.3181 | Alpha:0.4228 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:86.8500 | AUROC:0.0000
Test | 65/8 | Loss:0.1560 | MainLoss:0.1560 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:96.0374 | AUROC:0.9992
Test | 62/8 | Loss:0.2654 | MainLoss:0.2654 | SPLoss:0.0008 | CLSLoss:0.0692 | top1:89.2464 | AUROC:0.9906

Epoch: [294 | 1000] LR: 0.001306
Train | 8/8 